# Naloga razvrščanja besedila

Kot smo že omenili, se bomo osredotočili na preprosto nalogo razvrščanja besedila na podlagi podatkovnega nabora **AG_NEWS**, kjer razvrščamo naslove novic v eno od 4 kategorij: Svet, Šport, Posel in Znanost/Tehnologija.

## Podatkovni nabor

Ta podatkovni nabor je vgrajen v modul [`torchtext`](https://github.com/pytorch/text), zato imamo do njega enostaven dostop.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Tukaj `train_dataset` in `test_dataset` vsebujeta zbirke, ki vračajo pare oznake (številka razreda) in besedila, na primer:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Torej, natisnimo prvih 10 novih naslovov iz našega nabora podatkov:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Ker so podatkovne zbirke iteratorji, jih moramo, če želimo podatke uporabiti večkrat, pretvoriti v seznam:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenizacija

Zdaj moramo besedilo pretvoriti v **številke**, ki jih lahko predstavimo kot tenzorje. Če želimo predstavitev na ravni besed, moramo narediti dve stvari:
* uporabiti **tokenizator**, da razdelimo besedilo na **tokene**
* zgraditi **besedišče** teh tokenov.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Z uporabo besedišča lahko zlahka kodiramo našo tokenizirano niz v niz številk:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Predstavitev besedila z metodo Bag of Words

Ker besede predstavljajo pomen, lahko včasih ugotovimo pomen besedila zgolj z opazovanjem posameznih besed, ne glede na njihov vrstni red v stavku. Na primer, pri razvrščanju novic besede, kot so *vreme* in *sneg*, verjetno nakazujejo na *vremensko napoved*, medtem ko besede, kot so *delnice* in *dolar*, kažejo na *finančne novice*.

**Bag of Words** (BoW) je najpogosteje uporabljena tradicionalna predstavitev vektorskih podatkov. Vsaka beseda je povezana z indeksom vektorja, element vektorja pa vsebuje število pojavitev besede v določenem dokumentu.

![Slika prikazuje, kako je predstavitev vektorja Bag of Words shranjena v pomnilniku.](../../../../../translated_images/sl/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Na Bag of Words lahko gledate tudi kot na vsoto vseh vektorjev, kodiranih z metodo one-hot, za posamezne besede v besedilu.

Spodaj je primer, kako ustvariti predstavitev Bag of Words z uporabo knjižnice Scikit Learn v Pythonu:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Za izračun vektorja vreče besed iz vektorske predstavitve našega nabora podatkov AG_NEWS lahko uporabimo naslednjo funkcijo:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Opomba:** Tukaj uporabljamo globalno spremenljivko `vocab_size` za določitev privzete velikosti besedišča. Ker je velikost besedišča pogosto precej velika, lahko omejimo velikost besedišča na najpogostejše besede. Poskusite znižati vrednost `vocab_size` in zagnati spodnjo kodo ter opazujte, kako to vpliva na natančnost. Pričakujete lahko nekaj zmanjšanja natančnosti, vendar ne dramatičnega, v zameno za boljšo zmogljivost.


## Učenje BoW klasifikatorja

Zdaj, ko smo se naučili, kako zgraditi predstavitev Bag-of-Words za naš besedilni korpus, lahko na tej osnovi naučimo klasifikator. Najprej moramo naš nabor podatkov za učenje pretvoriti tako, da se vse pozicijske vektorske predstavitve spremenijo v predstavitev Bag-of-Words. To lahko dosežemo tako, da funkcijo `bowify` podamo kot parameter `collate_fn` standardnemu torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Zdaj definirajmo preprosto nevronsko mrežo za klasifikacijo, ki vsebuje eno linearno plast. Velikost vhodnega vektorja je enaka `vocab_size`, velikost izhoda pa ustreza številu razredov (4). Ker rešujemo nalogo klasifikacije, je končna aktivacijska funkcija `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Zdaj bomo definirali standardno učno zanko v PyTorchu. Ker je naš nabor podatkov precej velik, bomo za namen učenja trenirali le eno epoho, včasih pa celo manj kot eno epoho (določitev parametra `epoch_size` nam omogoča omejitev učenja). Prav tako bomo med učenjem poročali o akumulirani natančnosti učenja; pogostost poročanja je določena s parametrom `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrami, TriGrami in N-Grami

Ena od omejitev pristopa vreče besed je, da so nekatere besede del večbesednih izrazov. Na primer, beseda 'hot dog' ima popolnoma drugačen pomen kot besedi 'hot' in 'dog' v drugih kontekstih. Če besedi 'hot' in 'dog' vedno predstavljamo z istimi vektorji, lahko to zmede naš model.

Da bi to rešili, se pogosto uporabljajo **N-gramske reprezentacije** pri metodah za klasifikacijo dokumentov, kjer je frekvenca vsake besede, dvobesedne ali tribesedne kombinacije koristna značilnost za treniranje klasifikatorjev. Pri bigramski reprezentaciji, na primer, dodamo v besedišče vse pare besed poleg izvirnih besed.

Spodaj je primer, kako ustvariti bigramsko vrečo besed z uporabo Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Glavna pomanjkljivost pristopa N-gram je, da velikost besedišča začne izjemno hitro naraščati. V praksi moramo združiti predstavitev N-gram z nekaterimi tehnikami za zmanjšanje dimenzionalnosti, kot so *vgnezditve* (embeddings), o katerih bomo govorili v naslednji enoti.

Za uporabo predstavitve N-gram v našem naboru podatkov **AG News** moramo zgraditi posebno ngram besedišče:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Nato bi lahko uporabili isto kodo kot zgoraj za treniranje klasifikatorja, vendar bi bila to zelo neučinkovita uporaba pomnilnika. V naslednji enoti bomo trenirali bigram klasifikator z uporabo vdelav.

> **Opomba:** Pustite lahko samo tiste ngrame, ki se v besedilu pojavijo več kot določeno število krat. To bo zagotovilo, da bodo redki bigrami izpuščeni, in bo znatno zmanjšalo dimenzionalnost. Za to nastavite parameter `min_freq` na višjo vrednost in opazujte spremembo dolžine besedišča.


## Frekvenca izraza in obratna frekvenca dokumenta (TF-IDF)

V predstavitvi BoW so pojavitve besed enakovredno ovrednotene, ne glede na samo besedo. Vendar je jasno, da so pogoste besede, kot so *a*, *in* itd., veliko manj pomembne za klasifikacijo kot specializirani izrazi. Pravzaprav so pri večini nalog NLP nekatere besede bolj relevantne kot druge.

**TF-IDF** pomeni **frekvenca izraza–obratna frekvenca dokumenta**. Gre za različico vreče besed, kjer se namesto binarne vrednosti 0/1, ki označuje pojav besede v dokumentu, uporablja vrednost s plavajočo vejico, ki je povezana s frekvenco pojavljanja besede v korpusu.

Bolj formalno je teža $w_{ij}$ besede $i$ v dokumentu $j$ definirana kot:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kjer
* $tf_{ij}$ predstavlja število pojavitev $i$ v $j$, torej vrednost BoW, ki smo jo videli prej
* $N$ je število dokumentov v zbirki
* $df_i$ je število dokumentov, ki vsebujejo besedo $i$ v celotni zbirki

Vrednost TF-IDF $w_{ij}$ se povečuje sorazmerno s številom pojavitev besede v dokumentu in se zmanjša glede na število dokumentov v korpusu, ki vsebujejo to besedo, kar pomaga prilagoditi dejstvo, da se nekatere besede pojavljajo pogosteje kot druge. Na primer, če se beseda pojavi v *vsakem* dokumentu v zbirki, potem je $df_i=N$, in $w_{ij}=0$, takšni izrazi pa bi bili popolnoma prezrti.

TF-IDF vektorizacijo besedila lahko enostavno ustvarite s pomočjo Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Zaključek

Čeprav TF-IDF predstavitve besedam dodelijo uteži glede na njihovo pogostost, ne morejo predstaviti pomena ali vrstnega reda. Kot je slavni jezikoslovec J. R. Firth leta 1935 dejal: "Popoln pomen besede je vedno kontekstualen in nobena študija pomena brez konteksta ne more biti resna." Kasneje v tečaju se bomo naučili, kako zajeti kontekstualne informacije iz besedila z uporabo jezikovnega modeliranja.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
